In [22]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv('train.csv', parse_dates=['timestamp'], index_col=0)
data

,user_id,timestamp,gate_id
row_id,,,
0,18,2022-07-29 09:08:54,7
1,18,2022-07-29 09:09:54,9
2,18,2022-07-29 09:09:54,9
3,18,2022-07-29 09:10:06,5
4,18,2022-07-29 09:10:08,5
...,...,...,...
37513,6,2022-12-31 20:38:56,11
37514,6,2022-12-31 20:39:22,6
37515,6,2022-12-31 20:39:23,6


In [4]:
data_train = pd.read_csv('test.csv', parse_dates=['timestamp'], index_col=0)
data_train

,timestamp,gate_id
row_id,,
37518,2023-01-03 08:21:00,9
37519,2023-01-03 08:21:00,9
37520,2023-01-03 08:21:18,5
37521,2023-01-03 08:21:19,5
37522,2023-01-03 08:21:39,10
...,...,...
44638,2023-02-24 19:43:36,11
44639,2023-02-24 19:44:00,4
44640,2023-02-24 19:44:01,4


In [5]:
def dater(x):
    x=x.rename(columns={'timestamp':'ts'})
    x['hour'] = x['ts'].dt.hour
    x['min'] = x['ts'].dt.minute
    x['day'] = x['ts'].dt.day
    x['dayofweek'] = x['ts'].dt.dayofweek
    return x

In [6]:
data=dater(data)

In [7]:
data_train=dater(data_train)

In [8]:
data_valid=data.loc[data.sample(round(len(data)*0.2)).index]

In [ ]:
y=data.user_id
X=data.loc[:,'gate_id':]

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23, random_state=42)

In [41]:
stdsc= StandardScaler()
stdsc.fit(X_train)

X_train = stdsc.transform(X_train)
X_test=stdsc.transform(X_test)

In [12]:
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['saga'],
    'max_iter' : [1000]
    }
]

In [13]:
logModel = LogisticRegression()

In [14]:
clf = GridSearchCV(logModel, param_grid = param_grid, cv = 2, verbose=True, n_jobs=-1)

In [15]:
best_clf = clf.fit(X,y)

Fitting 2 folds for each of 80 candidates, totalling 160 fits


C:\Users\79618\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\79618\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\79618\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 1291, in fit
    fold_coefs_ = Parallel(n_jobs=self.n_jobs, verbose=self.verbose, prefer=prefer)(
  File "C:\Users\79618\AppDat

In [43]:
accuracy_score(y_test,best_clf.best_estimator_.predict(X_test))

C:\Users\79618\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


0.034414831981460024

In [19]:
best_clf.best_score_

0.0716722639799563

In [59]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint

print('Hyperparameters currently in use:\n')
pprint(rf.get_params())

Hyperparameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [64]:
penalty = ['l1', 'l2'] 
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000] 
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}] 
solver = ['liblinear', 'saga'] 
param_grid = dict(penalty=penalty, 
C=C, 
class_weight=class_weight, 
solver=solver) 
logistic = LogisticRegression() 
grid = GridSearchCV(estimator=logistic, 
param_grid=param_grid, 
scoring='accuracy', 
verbose=3, 
n_jobs=-1)
grid_result = grid.fit(X_train, y_train) 
print('Best Score: ', grid_result.best_score_) 
print('Best Params: ', grid_result.best_params_) 

TypeError: GridSearchCV.__init__() got an unexpected keyword argument 'n_splits'

In [58]:
lr=LogisticRegression(max_iter=10000,C=2, penalty='l2',solver='lbfgs', multi_class="auto")
m=lr.fit(X_train,y_train)
accuracy_score(y_test,m.predict(X_test))

0.08273464658169177

In [51]:
num_folds = 10
seed = 7
scoring = 'accuracy'
from pandas import read_csv
from pandas import set_option
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
penalty = ['l1', 'l2']
param_grid = dict(C=c_values, penalty=penalty)
model = LogisticRegression(max_iter=1000)
kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold, verbose=True)
grid_result = grid.fit(X_train, y_train)
print("Best: {} using {}".format(grid_result.best_score_, grid_result.best_params_))

Fitting 10 folds for each of 20 candidates, totalling 200 fits


C:\Users\79618\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
100 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\79618\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\79618\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\79618\AppData\Local\Programs\P

Best: 0.0792372731029629 using {'C': 0.1, 'penalty': 'l2'}
